In [15]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Model
from keras.layers import Input, Dense, Bidirectional, LSTM

In [17]:
df = pd.read_excel('../data/36_4_7.xlsx', header=[0, 1])

df.head(38)

Unnamed: 0_level_0 Unnamed: 1_level_0  \
               Mouse Category            Mouse #   
0      Group-1;Type-A;gradual                  1   
1          Group-2;Type-B;non                  2   
2          Group-2;Type-B;non                  3   
3          Group-2;Type-B;non                  4   
4          Group-2;Type-B;non                  5   
5          Group-2;Type-B;non                  6   
6        Group-3;Type-C;early                  7   
7          Group-2;Type-B;non                  8   
8          Group-2;Type-B;non                  9   
9          Group-2;Type-B;non                 10   
10         Group-2;Type-B;non                 11   
11         Group-2;Type-B;non                 12   
12       Group-3;Type-C;early                 13   
13       Group-3;Type-C;early                 14   
14         Group-2;Type-B;non                 15   
15         Group-2;Type-B;non                 16   
16         Group-2;Type-B;non                 17   
17     Group-1;Type-A;gradual                 18   
18         Group-2;Type-B;non                 19   
19         Group-2;Type-B;non                 20   
20         Group-2;Type-B;non                 21   
21         Group-2;Type-B;non                 22   
22         Group-2;Type-B;non                 23   
23       Group-3;Type-C;early                 24   
24         Group-2;Type-B;non                 25   
25         Group-2;Type-B;non                 26   
26         Group-2;Type-B;non                 27   
27  Group-1;Type-A;gradual#28                 28   
28         Group-2;Type-B;non                 29   
29     Group-1;Type-A;gradual                 30   
30       Group-3;Type-C;early                 31   
31     Group-1;Type-A;gradual                 32   
32         Group-2;Type-B;non                 33   
33         Group-2;Type-B;non                 34   
34         Group-2;Type-B;non                 35   
35         Group-2;Type-B;non                 36   

   Daily ethanol intake (g/kg/day)                                 \
                             Day 1      Day 2     Day 3     Day 4   
0                         2.428519   1.185000  1.273756  0.580301   
1                         1.634483   0.396416  0.521181  0.051634   
2                         2.842678   1.433402  2.197984  2.450099   
3                         1.602899   1.420863  2.268858  0.697453   
4                         0.405872   0.931023  0.187140  1.706710   
5                         1.577079   1.777153  1.323419  2.402645   
6                         1.908725   1.261345  1.691717  1.043107   
7                         1.706400   0.545873  1.517514  0.462157   
8                         2.190756   1.083065  2.296512  1.137121   
9                         1.866142   0.472523  0.712996  0.224113   
10                        0.497638   0.842667  0.611050  0.523757   
11                        0.725048   1.473881  0.551838  1.366486   
12                        1.102326   1.256818  0.726103  1.144928   
13                        1.115985   0.926007  0.871530  0.833040   
14                        0.968868   0.637668  0.309804  0.749138   
15                        1.409665   1.237011  0.027288  0.160678   
16                        0.155665   0.099215  0.000000  0.146977   
17                        1.065873   0.176208  0.258545  0.336767   
18                        1.122558   0.791319  0.387255  0.385366   
19                        1.253968   0.531092  2.227197  1.392628   
20                        0.872928   0.572464  0.682211  0.132550   
21                        1.002114   0.617188  0.000000  0.147388   
22                        1.190207   5.024735  0.133221  0.000000   
23                        7.492784  10.310940  7.544944  7.463458   
24                        0.669492   0.323108  0.475904  0.153101   
25                        0.871324   1.642981  1.438742  0.384740   
26                        0.916828   0.563280  0.685764  0.670628   
27                        1.81262

In [18]:
print(df.columns)

MultiIndex([(                                       'Unnamed: 0_level_0', ...),
            (                                       'Unnamed: 1_level_0', ...),
            (                          'Daily ethanol intake (g/kg/day)', ...),
            (                          'Daily ethanol intake (g/kg/day)', ...),
            (                          'Daily ethanol intake (g/kg/day)', ...),
            (                          'Daily ethanol intake (g/kg/day)', ...),
            (                                       'Daily total liquid', ...),
            (                                       'Daily total liquid', ...),
            (                                       'Daily total liquid', ...),
            (                                       'Daily total liquid', ...),
            (                                    'Daily food intake (g)', ...),
            (                                    'Daily food intake (g)', ...),
            (                           

Daily total liquid: These units represent the amount of liquid consumed by the subject on a daily basis. The unit is likely expressed in milliliters (ml) or another volume unit.

Daily food intake (g): These units represent the amount of food consumed by the subject on a daily basis. The unit is likely expressed in grams (g) or another weight unit.

Body weight (g): These units represent the body weight of the subject. The unit is likely expressed in grams (g) or another weight unit.

Ethanol preference (100*daily ethanol/daily total liquid): This column represents the ethanol preference of the subject, calculated as a percentage. It indicates how much of the daily liquid intake is attributed to ethanol consumption. The unit is percentage (%).

Daily liquid/weight (1000*ml/kg): This column represents the ratio of daily liquid intake to body weight. It is likely expressed as milliliters per kilogram (ml/kg).

Daily food intake/weight: This column represents the ratio of daily food intake to body weight. It is likely expressed as grams per kilogram (g/kg).

Mouse Category:
- Group 2: normal
- Group 1 and 3: alcoholic

## Make 3-d numpy array

Example of Day 1 Data with mouse 1 (Alcohol, food, water, and body weight) with 3 time steps:


| Mouse Number | Time Step 1 | Time Step 2 | Time Step 3 | Time Step 4| Features |
|--------------:|------------:|------------:|------------:|---------:| ---------:|
| 1             | [2.43, 4.15, 4.95, 27, 73.45, 209.26, 5.50] | [1.19, 2.10, 5.20, 28.00, 36.92, 189.2, 5.57] |  [1.27, 2.35, 5.30, 29.15, 51.09, 157.8, 5.45]    | [0.58, 1.10, 5.80, 29.95, 19.82, 185.31, 5.81]  |7|


Expected numpy array output: 

### Mouse #1:
- Time Step 1: [2.43, 4.15, 4.95, 27, 73.45, 209.26, 5.50]
- Time Step 2: [1.19, 2.10, 5.20, 28.00, 36.92, 189.2, 5.57]
- Time Step 3: [1.27, 2.35, 5.30, 29.15, 51.09, 157.8, 5.45]
- Time Step 4: [0.58, 1.10, 5.80, 29.95, 19.82, 185.31, 5.81]

In [19]:
daily_ethanol_columns = [('Daily ethanol intake (g/kg/day)', 'Day 1'), ('Daily ethanol intake (g/kg/day)', 'Day 2'), 
                         ('Daily ethanol intake (g/kg/day)', 'Day 3'), ('Daily ethanol intake (g/kg/day)', 'Day 4')]
daily_total_liquid_columns = [('Daily total liquid', 'Day 1'), ('Daily total liquid', 'Day 2'), 
                              ('Daily total liquid', 'Day 3'), ('Daily total liquid', 'Day 4')]
daily_food_intake_columns = [('Daily food intake (g)', 'Day 1'), ('Daily food intake (g)', 'Day 2'), 
                             ('Daily food intake (g)', 'Day 3'), ('Daily food intake (g)', 'Day 4')]
body_weight_columns = [('Body weight (g)', 'Day 1'), ('Body weight (g)', 'Day 2'), 
                       ('Body weight (g)', 'Day 3'), ('Body weight (g)', 'Day 4')]
ethanol_preference_columns = [('Ethanol preference (100*daily ethanol/daily total liquid)', 'Day 1'), ('Ethanol preference (100*daily ethanol/daily total liquid)', 'Day 2'), 
                              ('Ethanol preference (100*daily ethanol/daily total liquid)', 'Day 3'), ('Ethanol preference (100*daily ethanol/daily total liquid)', 'Day 4')]
daily_liquid_weight_columns = [('Daily liquid/weight (1000*ml/kg)', 'Day 1'), ('Daily liquid/weight (1000*ml/kg)', 'Day 2'), 
                               ('Daily liquid/weight (1000*ml/kg)', 'Day 3'), ('Daily liquid/weight (1000*ml/kg)', 'Day 4')]
daily_food_intake_weight_columns = [('Daily food intake/weight', 'Day 1'), ('Daily food intake/weight', 'Day 2'),
                                    ('Daily food intake/weight', 'Day 3'), ('Daily food intake/weight', 'Day 4')]

num_mice = len(df)
num_time_steps = 4
num_features = 7 

data_3d = np.zeros((num_mice, num_time_steps, num_features))

for i in range(num_mice):
    data_3d[i, :, 0] = df.iloc[i, df.columns.get_loc(daily_ethanol_columns[0]):df.columns.get_loc(daily_ethanol_columns[-1])+1].values
    data_3d[i, :, 1] = df.iloc[i, df.columns.get_loc(daily_total_liquid_columns[0]):df.columns.get_loc(daily_total_liquid_columns[-1])+1].values
    data_3d[i, :, 2] = df.iloc[i, df.columns.get_loc(daily_food_intake_columns[0]):df.columns.get_loc(daily_food_intake_columns[-1])+1].values
    data_3d[i, :, 3] = df.iloc[i, df.columns.get_loc(body_weight_columns[0]):df.columns.get_loc(body_weight_columns[-1])+1].values
    data_3d[i, :, 4] = df.iloc[i, df.columns.get_loc(ethanol_preference_columns[0]):df.columns.get_loc(ethanol_preference_columns[-1])+1].values
    data_3d[i, :, 5] = df.iloc[i, df.columns.get_loc(daily_liquid_weight_columns[0]):df.columns.get_loc(daily_liquid_weight_columns[-1])+1].values
    data_3d[i, :, 6] = df.iloc[i, df.columns.get_loc(daily_food_intake_weight_columns[0]):df.columns.get_loc(daily_food_intake_weight_columns[-1])+1].values
    
for i, sample in enumerate(data_3d):
    print(f"\n### Mouse #{i + 1}:")
    for t, time_step in enumerate(sample):
        print(f"Time Step {t + 1}: {time_step}")


### Mouse #1:
Time Step 1: [  2.42851852   4.15         4.95        27.          73.45132743
 209.25925926   5.5       ]
Time Step 2: [  1.185        2.1          5.2         28.          39.62264151
 189.28571429   5.57142857]
Time Step 3: [  1.27375643   2.35         5.3         29.15        51.08695652
 157.80445969   5.45454545]
Time Step 4: [  0.5803005    1.1          5.8         29.95        19.81981982
 185.30884808   5.8096828 ]

### Mouse #2:
Time Step 1: [  1.63448276   2.4          7.55        23.2         23.76237624
 435.34482759   9.76293103]
Time Step 2: [  0.39641577   0.7          5.85        27.9         11.66666667
 215.05376344   6.29032258]
Time Step 3: [  0.52118056   0.95         6.95        28.8         13.76811594
 239.58333333   7.23958333]
Time Step 4: [5.16339869e-02 1.00000000e-01 7.10000000e+00 3.06000000e+01
 1.29032258e+00 2.53267974e+02 6.96078431e+00]

### Mouse #3:
Time Step 1: [  2.84267782   4.3          5.45        23.9         75.43859649
 238.4

In [20]:
print(data_3d)

[[[2.42851852e+00 4.15000000e+00 4.95000000e+00 ... 7.34513274e+01
   2.09259259e+02 5.50000000e+00]
  [1.18500000e+00 2.10000000e+00 5.20000000e+00 ... 3.96226415e+01
   1.89285714e+02 5.57142857e+00]
  [1.27375643e+00 2.35000000e+00 5.30000000e+00 ... 5.10869565e+01
   1.57804460e+02 5.45454545e+00]
  [5.80300501e-01 1.10000000e+00 5.80000000e+00 ... 1.98198198e+01
   1.85308848e+02 5.80968280e+00]]

 [[1.63448276e+00 2.40000000e+00 7.55000000e+00 ... 2.37623762e+01
   4.35344828e+02 9.76293103e+00]
  [3.96415771e-01 7.00000000e-01 5.85000000e+00 ... 1.16666667e+01
   2.15053763e+02 6.29032258e+00]
  [5.21180556e-01 9.50000000e-01 6.95000000e+00 ... 1.37681159e+01
   2.39583333e+02 7.23958333e+00]
  [5.16339869e-02 1.00000000e-01 7.10000000e+00 ... 1.29032258e+00
   2.53267974e+02 6.96078431e+00]]

 [[2.84267782e+00 4.30000000e+00 5.45000000e+00 ... 7.54385965e+01
   2.38493724e+02 6.84100418e+00]
  [1.43340206e+00 2.20000000e+00 5.80000000e+00 ... 4.23076923e+01
   2.14432990e+02 7.

## Create a new column to signify alcholic factor

In [21]:
# Find the index level corresponding to 'Mouse Category' column
mouse_category_index = df.columns.get_loc(('Unnamed: 0_level_0', 'Mouse Category'))

# Define a function to map Mouse Category to 0 (Non-Alcoholic) or 1 (Alcoholic)
def map_alcoholic_to_binary(row):
    if 'Group-2' in row:
        return 0  # Non-Alcoholic
    else:
        return 1  # Alcoholic

# Create the new 'Alcoholic' column using the map_alcoholic_to_binary function
df['Alcoholic'] = df.iloc[:, mouse_category_index].apply(map_alcoholic_to_binary)


df.head(15)

Unnamed: 0_level_0 Unnamed: 1_level_0 Daily ethanol intake (g/kg/day)  \
            Mouse Category            Mouse #                           Day 1   
0   Group-1;Type-A;gradual                  1                        2.428519   
1       Group-2;Type-B;non                  2                        1.634483   
2       Group-2;Type-B;non                  3                        2.842678   
3       Group-2;Type-B;non                  4                        1.602899   
4       Group-2;Type-B;non                  5                        0.405872   
5       Group-2;Type-B;non                  6                        1.577079   
6     Group-3;Type-C;early                  7                        1.908725   
7       Group-2;Type-B;non                  8                        1.706400   
8       Group-2;Type-B;non                  9                        2.190756   
9       Group-2;Type-B;non                 10                        1.866142   
10      Group-2;Type-B;non                 11                        0.497638   
11      Group-2;Type-B;non                 12                        0.725048   
12    Group-3;Type-C;early                 13                        1.102326   
13    Group-3;Type-C;early                 14                        1.115985   
14      Group-2;Type-B;non                 15                        0.968868   

                                 Daily total liquid                    ...  \
       Day 2     Day 3     Day 4              Day 1 Day 2 Day 3 Day 4  ...   
0   1.185000  1.273756  0.580301               4.15   2.1  2.35   1.1  ...   
1   0.396416  0.521181  0.051634               2.40   0.7  0.95   0.1  ...   
2   1.433402  2.197984  2.450099               4.30   2.2  3.45   3.9  ...   
3   1.420863  2.268858  0.697453               2.80   2.5  4.15   1.3  ...   
4   0.931023  0.187140  1.706710               0.70   1.7  0.35   3.3  ...   
5   1.777153  1.323419  2.402645               2.70   3.2  2.45   4.6  ...   
6   1.261345  1.691717  1.043107               2.70   1.9  2.65   1.7  ...   
7   0.545873  1.517514  0.462157               2.70   0.9  2.55   0.8  ...   
8   1.083065  2.296512  1.137121               3.30   1.7  3.75   1.9  ...   
9   0.472523  0.712996  0.224113               3.00   0.8  1.25   0.4  ...   
10  0.842667  0.611050  0.523757               0.80   1.4  1.05   0.9  ...   
11  1.473881  0.551838  1.366486               1.20   2.5  0.95   2.4  ...   
12  1.256818  0.726103  1.144928               1.80   2.1  1.25   2.0  ...   
13  0.926007  0.871530  0.833040               1.90   1.6  1.55   1.5  ...   
14  0.637668  0.309804  0.749138               1.30   0.9  0.45   1.1  ...   

   Ethanol preference (100*daily ethanol/daily total liquid)  \
                                                       Day 4   
0                                           19.819820          
1                                            1.290323          
2                                           78.787879          
3                                           21.487603          
4                                           44.295302          
5                                           69.172932          
6                                           34.343434          
7                                           16.842105          
8                                           30.400000          
9                                            7.476636          
10                                          11.464968          
11                                          46.601942          
12                                          34.188034          
13                                          30.303030          
14                                          24.719101          

   Daily liquid/weight (1000*ml/kg)                                      \
                              Day 1       Day 2       Day 3       Day 4   
0                        209.259259  189.285714  157.804460  185

## Feed into Model

In [22]:
# Define model for simple BI-LSTM + DNN based binary classifier
def define_model(num_features):
    input1 = Input(shape=(num_time_steps, num_features))
    lstm1 = Bidirectional(LSTM(units=32))(input1)
    dnn_hidden_layer1 = Dense(3, activation='relu')(lstm1)
    dnn_output = Dense(1, activation='sigmoid')(dnn_hidden_layer1)
    model = Model(inputs=[input1], outputs=[dnn_output])

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model


Y = df['Alcoholic'].values  

num_mice, num_time_steps, num_features = data_3d.shape

model = define_model(num_features)

model.fit(data_3d, Y, epochs=4, batch_size=2, verbose=1)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 4, 7)]            0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                10240     
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 3)                 195       
                                                                 
 dense_3 (Dense)             (None, 1)                 4         
                                                                 
Total params: 10439 (40.78 KB)
Trainable params: 10439 (40.78 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/4
18/18 [==============================] - 1s 1ms/step - loss: 0.6924 - accuracy:

## Evaluating Model

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_3d, Y, test_size=0.2, random_state=42)

model = define_model()

model.fit(X_train, y_train, epochs=4, batch_size=2, verbose=1)

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)